In [1]:
import numpy as np
import pandas as pd
import random
import sys

In [4]:
import keras
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

In [2]:
df = pd.read_csv('Combined Reviews.csv')

In [3]:
df.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [19]:
df['Review'].value_counts()

Does not fit.                                                                        2
Great phone!.                                                                        2
Excellent product for the price.                                                     2
Works great!.                                                                        2
I would not recommend this place.                                                    2
                                                                                    ..
Talk about USELESS customer service.                                                 1
Good value, great food, great service.                                               1
Absolutely great.                                                                    1
Cheap but hey it works.. Was pleasantly suprised given the low cost of this item.    1
A PIECE OF JUNK THAT BROKE AFTER BEING ON MY PHONE FOR 2 DAYS!!!                     1
Name: Review, Length: 1986, dtype: int64

In [9]:
t = Tokenizer()
t.fit_on_texts(df['Review'])
text_mat = t.texts_to_sequences(df['Review'])

In [45]:
def find_vocab_size(matrix = text_mat):
    
    sequence_count = [set(text_mat[i])for i in range(len(text_mat))]
    total = []
    for i in range(len(sequence_count)):
        for j in sequence_count[i]:
            total.append(j)
    
    print(len(set(total))) #total number of unique words, or vocabulary of the text
    
    vocab = len(set(total))
    return vocab

In [11]:
len_mat = []
for i in range(len(text_mat)):
    len_mat.append(len(text_mat[i]))   

In [14]:
from statistics import mean, median
print("Maximum length of sequence : {}".format(max(len_mat)))
print("Average length of sequence : {}".format(mean(len_mat)))
print("Median length of sequence : {}".format(median(len_mat)))

Maximum length of sequence : 32
Average length of sequence : 10.598
Median length of sequence : 9.0


In [15]:
text_padding = sequence.pad_sequences(text_mat, maxlen=32, padding='post')

In [47]:
text_padding[0].shape

(32,)

In [64]:
def lstm_model():
    
    import keras
    from keras.models import Sequential
    from keras.layers import Dense, Activation, LSTM, Embedding
    
    model = Sequential()
#     model.add(Dense(32,input_shape=(text_padding[0].shape)))
#     model.add(Embedding(input_dim = find_vocab_size(text_mat)+1, output_dim=32, input_length = text_padding[0].shape, embeddings_regularizer= keras.regularizers.l2(0.001)))
    model.add(Embedding(find_vocab_size()+1,128))
    model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
    
    return model

In [71]:
base_lstm_model = lstm_model()

3258


In [72]:
features, target = text_padding, np.array(df['Sentiment'])

In [73]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', patience=5)
ckpt = ModelCheckpoint('weights.hdf5', save_best_only=True, save_weights_only=True, monitor='val_accuracy')

In [75]:
ep, bs = 20, 16
base_lstm_model.fit(features,target, validation_split=0.2, epochs=ep, batch_size=bs, callbacks=[es, ckpt])
NAME  = 'base_lstm_model_{}_ep_{}_bs'.format(ep,bs)
base_lstm_model.save(NAME)

Train on 1600 samples, validate on 400 samples
Epoch 1/20
1600/1600 [==============================] - 2s 1ms/step - loss: 0.0736 - accuracy: 0.9806 - val_loss: 0.6445 - val_accuracy: 0.8000
Epoch 2/20
1600/1600 [==============================] - 2s 1ms/step - loss: 0.0423 - accuracy: 0.9900 - val_loss: 1.0090 - val_accuracy: 0.7850
Epoch 3/20
1600/1600 [==============================] - 2s 1ms/step - loss: 0.0599 - accuracy: 0.9875 - val_loss: 1.2349 - val_accuracy: 0.7650
Epoch 4/20
1600/1600 [==============================] - 2s 1ms/step - loss: 0.0534 - accuracy: 0.9894 - val_loss: 0.8869 - val_accuracy: 0.7900
Epoch 5/20
1600/1600 [==============================] - 3s 2ms/step - loss: 0.0382 - accuracy: 0.9925 - val_loss: 1.0680 - val_accuracy: 0.7750
Epoch 6/20
1600/1600 [==============================] - 3s 2ms/step - loss: 0.0815 - accuracy: 0.9806 - val_loss: 0.9143 - val_accuracy: 0.7725
